# Inverse Problems Exercises: 2022s s11 (all)
https://www.umm.uni-heidelberg.de/miism/

## Notes
* Please **DO NOT** change the name of the `.ipynb` file. 
* Please **DO NOT** import extra packages to solve the tasks. 
* Please put the `.ipynb` file directly into the `.zip` archive without any intermediate folder. 

## Please provide your personal information
* full name (Name): 

* Damjan Kalšan

## C02: Subspace pursuit

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import sys

# from the nbgrader folders
sys.path.append('../../exercises')
sys.path.append('../../../exercises')

file_gaussian = 'file_gaussian.npz'
with np.load(file_gaussian) as data:
    f_true = data['f_true']
    A_psf = data['A_psf']

### Imaging model
The imaging model can be represented by
$$
g = h \otimes f_\text{true}
= Af_\text{true}
= \mathcal{F}^{-1}\{ \mathcal{F}\{h\} \mathcal{F}\{f_\text{true}\} \},
$$
$$
g' = g + \epsilon.
$$
* $f_\text{true}$ is the input signal
* $h$ is the point spread function (kernel)
* $\otimes$ is the convolution operator
* $A$ is the Toeplitz matrix of $h$
* $\mathcal{F}$ and $\mathcal{F}^{-1}$ are the Fourier transform operator and inverse Fourier transform operator
* $\epsilon$ is the additive Gaussian noise
* $g$ is the filtered signal
* $g'$ is the noisy signal

In [ ]:
def compute_haar_wavelet_matrix(dim):
    """ Create Haar wavelet transformation matrix H for the
    matrix vector multiplication implementation of Haar wavelet transformation.
    This function uses the following formula to create the Haar transformation matrix:
        H_n = 1/sqrt(2)[H_(n/2) kron(1 1), I_(n/2) kron (1 -1)]
    where 'kron' denotes the kronecker product.
    The iteration starts with H_1=[1]. The normalization constant 1/sqrt(2) ensure that
    H_n^T * H_n = I, where I is identity matrix.
    
    Note: Haar wavelets are the rows of H.
    
    :param dim: dimension of square Haar wavelet transformbation matrix..
    :return: H: Haar transformation matrix with size as the power of 2 - i.e. 2, 4, 8, 16, 32, etc.
    """

    level = int(np.log2(dim))
    if (2**level < dim):
        print('error: value of input parameter is not power of 2')
        return

    # lowpass filter
    lp = np.array([[1, 1]])
    # highpass filter
    hp = np.array([[1, -1]])
    # normalizazion constant
    NC = 1/np.sqrt(2)

    H = np.array([1])

    for i in range(level):
        t1 = np.kron(H, lp)
        t2 = np.kron(np.identity(len(H)),hp)
        T = np.concatenate((t1, t2), axis=0)
        H = np.dot(NC,T)

    return H

### Subspace multiplication
The multiplication of matrix $A$ and vector $b$ in the subspace $T$ is denoted as $A_T b$, where
$$
(A_T b)_i =
\begin{cases}
    0 & i \notin T \\
    (Ab)_i & otherwise \\
\end{cases}.
$$
* Given the matrix $A$
* Given the vector $b$
* Given the index array $T$
* Implement the function `submul()` (using `numpy.array`)

In [ ]:
def submul(A, b, T):
    """ Subspace multiplication.
    
    :param A: 2d matrix A.
    :param b: 1d vector b.
    :param T: Index array.
    :return: Subspace multiplication vector by definition.
    """
    
    T = list(T)
    res = np.zeros(A.shape[0], dtype=A.dtype)
    
    if len(T) > 0:
        res[T] = A[T, :] @ b
    
    return res

In [ ]:
# This cell contains hidden tests.

submul_TEST = submul(A_psf, f_true, [1, 3])


### Subspace pursuit
To find the sparse representation of the signal, the problem is formulated as follows
$$
\arg\min_s \|s\|_0,\ \text{s. t.}\ g = Af = A\Psi s = \Phi s,
$$
where $\Psi$ is the transform into the sparse space, and $\Phi = A\Psi$.

The solution of $s$ can be calculated by the subspace pursuit method
$$
\begin{align*}
T^{(0)} &= \{\} \\
s^{(0)} &= 0 \\
r^{(0)} &= g - \Phi s^{(0)} = g \\
\delta s^{(n)} &= \Phi^{PI} r^{(n-1)} = \Phi^{PI} g - s^{(n-1)} \\
\tilde T^{(n)} &= T^{(n-1)} \cup \{k\ \text{indices corresponding to the largest magnitude entries in}\ \delta s^{(n)}\} \\
\tilde s^{(n)} &= \Phi^{PI}_{\tilde T^{(n)}} g \\
T^{(n)} &= \{k\ \text{indices corresponding to the largest magnitude entries in}\ \tilde s^{(n)}\} \\
s^{(n)} &= \Phi^{PI}_{T^{(n)}} g \\
r^{(n)} &= g - \Phi s^{(n)} \\
\end{align*}
$$
with $\Phi^{PI} = (\Phi^T \Phi)^{-1} \Phi^T$. (using `numpy.linalg.pinv()`)

* Given the system matrix $A$
* Given the filtered signal $g$
* Given the transform $\Psi$
* Given the expected number of non-zero entries for the sparse problem $k$
* Given the number of iterations $n$
* Return the final value $s^{(n)}$ as the first output
* Return the history array of residual norm $[||r^{(0)}||, ..., ||r^{(n)}||]$ as the second output
* Implement the function `solve_subspace_pursuit()` (using `numpy.array`)

In [ ]:
def solve_subspace_pursuit(A, g, Psi, k, n):
    """ Restore the signal using Matching Pursuit Algorithm.
    
    :param A: 2d matrix A of the linear problem.
    :param g: Observed signal.
    :param Psi: Transform matrix.
    :param k: Expected number of non-zero entries for the sparse problem.
    :param n: Number of iterations.
    :return: Final s estimate and the history of residual norm
    """
    
    Phi = A @ Psi
    Phi_pinv = np.linalg.pinv(Phi)
    
    T = set()
    s = np.zeros(Phi.shape[1])
    r = g
    history = np.array([np.linalg.norm(r)])
    
    if k == 0:
        return s, history.repeat(n+1)
    
    for i in range(n):
        delta_s = Phi_pinv @ g - s
        T_tilda = T.union(set(np.argpartition(np.abs(delta_s), -k)[-k:]))
        s_tilda = submul(Phi_pinv, g, T_tilda)
        
        T = set(np.argpartition(np.abs(s_tilda), -k)[-k:])
        s = submul(Phi_pinv, g, T)
        r = g - Phi @ s
        
        history = np.append(history, np.linalg.norm(r))
        
    return s, history

### Signal processing 1
Process the signals as follows
* `A_1`
  - Pad zeros to the end of `A_psf` in the columns only
  - Make the size of `A_1` the exponent of next higher power of 2 in the columns only
* `g_1`: filtered signal of `A_psf` and `f_true`
* `Psi_1`: the corresponding tranform matrix (using `compute_haar_wavelet_matrix()`)
* `k_1`: 100 non-zero entries
* `n_1`: 10 iterations
* `s_1`: the estimate of the sparse representation
* `list_r_norm_1`: the history of residual norm

Display the result
* Plot the outputs in the subplots of `axs`
* In the first subplot, show the estimated input signal $f = \Psi s$ and the true input signal
* In the second subplot, show the estimated measurement $Af$ and the actual measurement
* In the third subplot, show the sparse representation $s$
* In the fourth subplot, show the history of residual norm
* Show the legend in each subplot

In [ ]:
############################ NOTE TO TUTOR ############################
# For this and the following subtask, I transposed the generated      #
# haar wavelet matrix, in order to put the basis vectors as columns   #
# of Psi, which is in line with the formulas provided for Matching    #
# pursuit.                                                            #
#                                                                     #
# I mention this, because it might just happen that you               #
# will assert for a non-transposed matrix.                            #
#######################################################################


def pad_matrix(A, rows=True, cols=True):
    power0 = int(np.ceil(np.log2(A.shape[0])))
    power1 = int(np.ceil(np.log2(A.shape[1])))
    pad0 = 2 ** power0 - A.shape[0] if rows else 0
    pad1 = 2 ** power1 - A.shape[1] if cols else 0
    return np.pad(A, ((0, pad0), (0, pad1)))


A_1 = pad_matrix(A_psf, rows=False, cols=True)
g_1 = A_psf @ f_true
Psi_1 = compute_haar_wavelet_matrix(A_1.shape[1]).T
k_1 = 100
n_1 = 10
s_1, list_r_norm_1 = solve_subspace_pursuit(A_1, g_1, Psi_1, k_1, n_1)


fig, axs = plt.subplots(4, 1, figsize=(15, 10))
fig.suptitle('Signal processing 1')

axs[0].plot(Psi_1 @ s_1, label="Estimated input signal $f_{estimated} = \Psi s$")
axs[0].plot(f_true, label="True input signal $f_{true}$")
axs[0].legend()

axs[1].plot(A_1 @ Psi_1 @ s_1, label="Estimated meassurement $g_{estimated} = A f_{estimated}$")
axs[1].plot(g_1, label="True meassurement $g_{true}$")
axs[1].legend()

axs[2].plot(s_1, label="Sparse representation $s=\Psi^{-1} f$")
axs[2].legend()

axs[3].plot(list_r_norm_1, label="Residual norm history")
axs[3].legend()

plt.show()

In [ ]:
# This cell contains hidden tests.

shape_A_1_TEST = A_1.shape

np.testing.assert_array_equal(shape_A_1_TEST[0], A_psf.shape[0])
np.testing.assert_array_equal(2**np.uint(np.log2(shape_A_1_TEST[1])), shape_A_1_TEST[1])


In [ ]:
# This cell contains hidden tests.


In [ ]:
# This cell contains hidden tests.


In [ ]:
# This cell contains hidden tests.


In [ ]:
# This cell contains hidden tests.

np.testing.assert_array_equal(list_r_norm_1.size, 11)


### Signal processing 2
Process the signals as follows
* Use `A_1`, `g_1`, `Psi_1`, `n_1`
* `list_k`: 100 numbers from 0 to 99
* `output_s`: a matrix of the estimated sparse representation, with each column corresponding to each $k$
* `output_r`: an array of the residual norm corresponding to each estimated sparse representation

Display the result
* Plot the outputs in the subplots of `axs`
* In the first subplot, show the first 100 rows of `output_s` as an image
* In the second subplot, show the residual norm as a function of $k$
* Add proper titles to the subplots

In [ ]:
list_k = np.arange(0, 100)

output_s = []
output_r = []
for k in list_k:
    s_k, list_r_norm_k = solve_subspace_pursuit(A_1, g_1, Psi_1, k, n_1)
    output_s.append(s_k)
    output_r.append(list_r_norm_k[-1])
    
output_s = np.array(output_s).T
output_r = np.array(output_r).T
    
fig, axs = plt.subplots(2, 1, figsize=(15, 10))
fig.suptitle('Signal processing 2')

axs[0].imshow(output_s[:100, :])
axs[0].set_xlabel("Number of sparse elements $K$")
axs[0].set_ylabel("Sparse representation $s$")
axs[0].set_title("Sparse representation $s$ for different maximum number of sparse elements $K$")

axs[1].plot(output_r)
axs[1].set_xlabel("Number of sparse elements $K$")
axs[1].set_ylabel("Residual norm $L_{2}$")
axs[1].set_title("Residual norm $L_{2}$ for different maximum number of sparse elements $K$")

plt.show()

In [ ]:
# This cell contains hidden tests.

np.testing.assert_array_equal(output_s.shape[1], 100)


In [ ]:
# This cell contains hidden tests.

np.testing.assert_array_equal(output_r.size, 100)
